# Module 5: Advanced Long-Term Memory Patterns

**Building on Previous Modules:**
- Module 5.3: LangGraph Store API
- Module 5.4: LangChain memory backends
- Module 5.5: **Production-ready patterns and advanced techniques!**

**What you'll learn:**
- 🗄️ PostgreSQL and MongoDB backends
- 🔍 Advanced search and filtering
- ♻️ Memory lifecycle management
- 🏢 Multi-tenant architecture
- 📊 Memory analytics
- ⚡ Performance optimization
- 🛡️ Security and access control
- 🎯 Production deployment patterns

**Real-World Scenarios:**
- Enterprise HR system with thousands of users
- Multi-tenant SaaS application
- High-performance chat systems
- Compliance and data retention

**Time:** 3-4 hours

## Setup: Install Dependencies

In [ ]:
# Install required packages
!pip install --pre -U langchain langchain-openai langgraph
!pip install psycopg2-binary  # PostgreSQL
!pip install pymongo  # MongoDB
!pip install sqlalchemy  # ORM support
!pip install pandas  # Analytics
!pip install faiss-cpu  # Vector search

## Setup: Imports

In [ ]:
from google.colab import userdata
import os
import json
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, field
import uuid

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import HumanMessage, SystemMessage

print("✅ Setup complete!")

---
# Part 1: PostgreSQL Backend 🗄️

**Production Pattern:** Use PostgreSQL for scalable, reliable memory storage

## Lab 1.1: PostgreSQL Memory Store

In [ ]:
import sqlite3  # Using SQLite for demo; same concepts apply to PostgreSQL
from contextlib import contextmanager

class PostgreSQLMemoryStore:
    """PostgreSQL-backed memory store.
    
    For actual PostgreSQL, use:
    import psycopg2
    conn = psycopg2.connect("dbname=mydb user=postgres password=secret")
    """
    
    def __init__(self, db_path: str = ":memory:"):
        self.db_path = db_path
        self._init_db()
    
    def _init_db(self):
        """Initialize database schema."""
        with self._get_connection() as conn:
            cursor = conn.cursor()
            
            # Memory table with indexes
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS memory_store (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    namespace TEXT NOT NULL,
                    key TEXT NOT NULL,
                    value TEXT NOT NULL,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    accessed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    access_count INTEGER DEFAULT 0,
                    UNIQUE(namespace, key)
                )
            """)
            
            # Indexes for performance
            cursor.execute("""
                CREATE INDEX IF NOT EXISTS idx_namespace 
                ON memory_store(namespace)
            """)
            
            cursor.execute("""
                CREATE INDEX IF NOT EXISTS idx_updated_at 
                ON memory_store(updated_at)
            """)
            
            conn.commit()
    
    @contextmanager
    def _get_connection(self):
        """Context manager for database connections."""
        conn = sqlite3.connect(self.db_path)
        conn.row_factory = sqlite3.Row
        try:
            yield conn
        finally:
            conn.close()
    
    def put(self, namespace: Tuple[str, ...], key: str, value: Dict[str, Any]):
        """Save or update memory item."""
        ns_str = ":".join(namespace)
        value_json = json.dumps(value)
        
        with self._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                INSERT INTO memory_store (namespace, key, value, updated_at)
                VALUES (?, ?, ?, CURRENT_TIMESTAMP)
                ON CONFLICT(namespace, key) DO UPDATE SET
                    value = excluded.value,
                    updated_at = CURRENT_TIMESTAMP
            """, (ns_str, key, value_json))
            
            conn.commit()
    
    def get(self, namespace: Tuple[str, ...], key: str) -> Optional[Dict[str, Any]]:
        """Retrieve memory item."""
        ns_str = ":".join(namespace)
        
        with self._get_connection() as conn:
            cursor = conn.cursor()
            
            # Update access tracking
            cursor.execute("""
                UPDATE memory_store 
                SET accessed_at = CURRENT_TIMESTAMP,
                    access_count = access_count + 1
                WHERE namespace = ? AND key = ?
            """, (ns_str, key))
            
            cursor.execute("""
                SELECT * FROM memory_store 
                WHERE namespace = ? AND key = ?
            """, (ns_str, key))
            
            row = cursor.fetchone()
            conn.commit()
            
            if row:
                return {
                    "value": json.loads(row["value"]),
                    "created_at": row["created_at"],
                    "updated_at": row["updated_at"],
                    "accessed_at": row["accessed_at"],
                    "access_count": row["access_count"]
                }
            return None
    
    def search(self, namespace: Tuple[str, ...]) -> List[Dict[str, Any]]:
        """Search by namespace prefix."""
        ns_str = ":".join(namespace)
        
        with self._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT * FROM memory_store 
                WHERE namespace LIKE ?
                ORDER BY updated_at DESC
            """, (f"{ns_str}%",))
            
            results = []
            for row in cursor.fetchall():
                results.append({
                    "namespace": row["namespace"],
                    "key": row["key"],
                    "value": json.loads(row["value"]),
                    "updated_at": row["updated_at"]
                })
            
            return results
    
    def delete(self, namespace: Tuple[str, ...], key: str) -> bool:
        """Delete memory item."""
        ns_str = ":".join(namespace)
        
        with self._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                DELETE FROM memory_store 
                WHERE namespace = ? AND key = ?
            """, (ns_str, key))
            
            deleted = cursor.rowcount > 0
            conn.commit()
            return deleted
    
    def get_stats(self) -> Dict[str, Any]:
        """Get memory store statistics."""
        with self._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("SELECT COUNT(*) as total FROM memory_store")
            total = cursor.fetchone()["total"]
            
            cursor.execute("""
                SELECT namespace, COUNT(*) as count 
                FROM memory_store 
                GROUP BY namespace
            """)
            by_namespace = {row["namespace"]: row["count"] for row in cursor.fetchall()}
            
            cursor.execute("""
                SELECT AVG(access_count) as avg_access 
                FROM memory_store
            """)
            avg_access = cursor.fetchone()["avg_access"] or 0
            
            return {
                "total_items": total,
                "by_namespace": by_namespace,
                "avg_access_count": avg_access
            }

# Test PostgreSQL-style store
pg_store = PostgreSQLMemoryStore()

print("=" * 70)
print("Lab 1.1: PostgreSQL Memory Store")
print("=" * 70 + "\n")

# Save employee profiles
pg_store.put(
    namespace=("acme", "employees"),
    key="user_101",
    value={"name": "Priya Sharma", "dept": "Engineering", "role": "Senior Dev"}
)

pg_store.put(
    namespace=("acme", "employees"),
    key="user_102",
    value={"name": "Rahul Verma", "dept": "Marketing", "role": "Manager"}
)

print("✅ Saved 2 employee profiles")

# Retrieve with metadata
data = pg_store.get(("acme", "employees"), "user_101")
print(f"\n📥 Retrieved: {data['value']['name']}")
print(f"   Created: {data['created_at']}")
print(f"   Access count: {data['access_count']}")

# Get statistics
stats = pg_store.get_stats()
print(f"\n📊 Store Statistics:")
print(f"   Total items: {stats['total_items']}")
print(f"   By namespace: {stats['by_namespace']}")

print("\n✅ PostgreSQL-style memory store with metadata tracking!")

---
# Part 2: Memory Lifecycle Management ♻️

**Critical for Production:** Manage memory growth, cleanup, archival

## Lab 2.1: Automatic Memory Cleanup

In [ ]:
class MemoryLifecycleManager:
    """Manages memory lifecycle: cleanup, archival, retention policies."""
    
    def __init__(self, store: PostgreSQLMemoryStore):
        self.store = store
    
    def cleanup_old_memories(self, days: int = 90):
        """Delete memories older than specified days."""
        cutoff_date = (datetime.now() - timedelta(days=days)).isoformat()
        
        with self.store._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT COUNT(*) as count FROM memory_store
                WHERE updated_at < ?
            """, (cutoff_date,))
            
            count_before = cursor.fetchone()["count"]
            
            cursor.execute("""
                DELETE FROM memory_store 
                WHERE updated_at < ?
            """, (cutoff_date,))
            
            deleted = cursor.rowcount
            conn.commit()
            
            return deleted
    
    def archive_inactive_memories(self, days: int = 30):
        """Archive memories not accessed in specified days."""
        cutoff_date = (datetime.now() - timedelta(days=days)).isoformat()
        
        with self.store._get_connection() as conn:
            cursor = conn.cursor()
            
            # Find inactive memories
            cursor.execute("""
                SELECT * FROM memory_store
                WHERE accessed_at < ?
            """, (cutoff_date,))
            
            archived = []
            for row in cursor.fetchall():
                archived.append({
                    "namespace": row["namespace"],
                    "key": row["key"],
                    "value": row["value"],
                    "archived_at": datetime.now().isoformat()
                })
            
            # In production: Save to archive table or cold storage
            # cursor.execute("INSERT INTO memory_archive ...")
            
            # Delete from active memory
            cursor.execute("""
                DELETE FROM memory_store 
                WHERE accessed_at < ?
            """, (cutoff_date,))
            
            conn.commit()
            
            return len(archived)
    
    def cleanup_by_access_pattern(self, min_access_count: int = 1):
        """Remove memories that haven't been accessed enough."""
        with self.store._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                DELETE FROM memory_store 
                WHERE access_count < ?
                AND created_at < datetime('now', '-7 days')
            """, (min_access_count,))
            
            deleted = cursor.rowcount
            conn.commit()
            
            return deleted
    
    def get_memory_health_report(self) -> Dict[str, Any]:
        """Generate health report for memory system."""
        with self.store._get_connection() as conn:
            cursor = conn.cursor()
            
            # Total memories
            cursor.execute("SELECT COUNT(*) as total FROM memory_store")
            total = cursor.fetchone()["total"]
            
            # Old memories (>90 days)
            cutoff = (datetime.now() - timedelta(days=90)).isoformat()
            cursor.execute("""
                SELECT COUNT(*) as old FROM memory_store
                WHERE updated_at < ?
            """, (cutoff,))
            old = cursor.fetchone()["old"]
            
            # Inactive memories (>30 days no access)
            inactive_cutoff = (datetime.now() - timedelta(days=30)).isoformat()
            cursor.execute("""
                SELECT COUNT(*) as inactive FROM memory_store
                WHERE accessed_at < ?
            """, (inactive_cutoff,))
            inactive = cursor.fetchone()["inactive"]
            
            # Low access memories
            cursor.execute("""
                SELECT COUNT(*) as low_access FROM memory_store
                WHERE access_count < 2
                AND created_at < datetime('now', '-7 days')
            """)
            low_access = cursor.fetchone()["low_access"]
            
            return {
                "total_memories": total,
                "old_memories_90d": old,
                "inactive_30d": inactive,
                "low_access": low_access,
                "health_score": self._calculate_health_score(total, old, inactive, low_access)
            }
    
    def _calculate_health_score(self, total, old, inactive, low_access) -> str:
        """Calculate overall health score."""
        if total == 0:
            return "N/A"
        
        cleanup_needed_pct = ((old + inactive + low_access) / total) * 100
        
        if cleanup_needed_pct < 10:
            return "Excellent"
        elif cleanup_needed_pct < 25:
            return "Good"
        elif cleanup_needed_pct < 50:
            return "Fair - Cleanup Recommended"
        else:
            return "Poor - Cleanup Required"

# Test lifecycle management
lifecycle_mgr = MemoryLifecycleManager(pg_store)

print("=" * 70)
print("Lab 2.1: Memory Lifecycle Management")
print("=" * 70 + "\n")

# Get health report
report = lifecycle_mgr.get_memory_health_report()
print("📊 Memory Health Report:")
print(f"   Total memories: {report['total_memories']}")
print(f"   Old (>90 days): {report['old_memories_90d']}")
print(f"   Inactive (>30 days): {report['inactive_30d']}")
print(f"   Low access: {report['low_access']}")
print(f"   Health score: {report['health_score']}")

print("\n✅ Memory lifecycle management implemented!")
print("💡 Run cleanup jobs regularly (e.g., daily cron job)")

---
# Part 3: Multi-Tenant Architecture 🏢

**Enterprise Pattern:** Isolated memory per organization/tenant

## Lab 3.1: Multi-Tenant Memory System

In [ ]:
@dataclass
class Tenant:
    """Represents an organization/tenant."""
    tenant_id: str
    name: str
    plan: str  # 'free', 'pro', 'enterprise'
    max_memories: int
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())

class MultiTenantMemorySystem:
    """Memory system with tenant isolation and quotas."""
    
    def __init__(self, store: PostgreSQLMemoryStore):
        self.store = store
        self.tenants: Dict[str, Tenant] = {}
    
    def register_tenant(self, tenant: Tenant):
        """Register a new tenant."""
        self.tenants[tenant.tenant_id] = tenant
    
    def get_tenant_namespace(self, tenant_id: str, *parts: str) -> Tuple[str, ...]:
        """Get namespaced key for tenant."""
        return (tenant_id, *parts)
    
    def put(self, tenant_id: str, namespace: Tuple[str, ...], 
            key: str, value: Dict[str, Any]) -> bool:
        """Save memory with tenant isolation and quota check."""
        # Check tenant exists
        if tenant_id not in self.tenants:
            raise ValueError(f"Tenant {tenant_id} not registered")
        
        tenant = self.tenants[tenant_id]
        
        # Check quota
        current_count = self._get_tenant_memory_count(tenant_id)
        if current_count >= tenant.max_memories:
            raise ValueError(f"Tenant {tenant_id} exceeded memory quota ({tenant.max_memories})")
        
        # Save with tenant namespace
        full_namespace = self.get_tenant_namespace(tenant_id, *namespace)
        self.store.put(full_namespace, key, value)
        
        return True
    
    def get(self, tenant_id: str, namespace: Tuple[str, ...], 
            key: str) -> Optional[Dict[str, Any]]:
        """Get memory with tenant isolation."""
        full_namespace = self.get_tenant_namespace(tenant_id, *namespace)
        return self.store.get(full_namespace, key)
    
    def search(self, tenant_id: str, namespace: Tuple[str, ...]) -> List[Dict[str, Any]]:
        """Search within tenant namespace only."""
        full_namespace = self.get_tenant_namespace(tenant_id, *namespace)
        return self.store.search(full_namespace)
    
    def _get_tenant_memory_count(self, tenant_id: str) -> int:
        """Get memory count for tenant."""
        results = self.store.search((tenant_id,))
        return len(results)
    
    def get_tenant_usage(self, tenant_id: str) -> Dict[str, Any]:
        """Get usage statistics for tenant."""
        if tenant_id not in self.tenants:
            return {"error": "Tenant not found"}
        
        tenant = self.tenants[tenant_id]
        current_count = self._get_tenant_memory_count(tenant_id)
        
        return {
            "tenant_id": tenant_id,
            "tenant_name": tenant.name,
            "plan": tenant.plan,
            "memories_used": current_count,
            "memories_limit": tenant.max_memories,
            "usage_percentage": (current_count / tenant.max_memories * 100) if tenant.max_memories > 0 else 0,
            "quota_remaining": tenant.max_memories - current_count
        }
    
    def list_all_tenants_usage(self) -> List[Dict[str, Any]]:
        """Get usage for all tenants."""
        return [self.get_tenant_usage(tid) for tid in self.tenants.keys()]

# Create multi-tenant system
mt_system = MultiTenantMemorySystem(pg_store)

# Register tenants
mt_system.register_tenant(Tenant(
    tenant_id="acme_corp",
    name="Acme Corporation",
    plan="enterprise",
    max_memories=10000
))

mt_system.register_tenant(Tenant(
    tenant_id="startup_xyz",
    name="Startup XYZ",
    plan="pro",
    max_memories=1000
))

print("=" * 70)
print("Lab 3.1: Multi-Tenant Memory System")
print("=" * 70 + "\n")

# Tenant 1: Save data
mt_system.put(
    tenant_id="acme_corp",
    namespace=("employees",),
    key="user_101",
    value={"name": "Priya", "dept": "Engineering"}
)
print("✅ Saved data for Acme Corp")

# Tenant 2: Save data
mt_system.put(
    tenant_id="startup_xyz",
    namespace=("employees",),
    key="user_201",
    value={"name": "Alex", "dept": "Product"}
)
print("✅ Saved data for Startup XYZ")

# Check isolation - Acme can't see Startup data
acme_data = mt_system.search("acme_corp", ("employees",))
print(f"\n🔒 Acme Corp employees: {len(acme_data)}")
for item in acme_data:
    print(f"   - {item['value']['name']}")

# Usage statistics
print("\n📊 Tenant Usage:")
for usage in mt_system.list_all_tenants_usage():
    print(f"   {usage['tenant_name']} ({usage['plan']}):")
    print(f"      Used: {usage['memories_used']}/{usage['memories_limit']}")
    print(f"      Usage: {usage['usage_percentage']:.1f}%")

print("\n✅ Multi-tenant system with isolation and quotas!")

---
# Part 4: Memory Analytics 📊

**Production Insights:** Monitor and optimize memory usage

## Lab 4.1: Memory Analytics Dashboard

In [ ]:
class MemoryAnalytics:
    """Analytics and monitoring for memory system."""
    
    def __init__(self, store: PostgreSQLMemoryStore):
        self.store = store
    
    def get_top_accessed_memories(self, limit: int = 10) -> List[Dict[str, Any]]:
        """Get most frequently accessed memories."""
        with self.store._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT namespace, key, value, access_count
                FROM memory_store
                ORDER BY access_count DESC
                LIMIT ?
            """, (limit,))
            
            results = []
            for row in cursor.fetchall():
                results.append({
                    "namespace": row["namespace"],
                    "key": row["key"],
                    "value": json.loads(row["value"]),
                    "access_count": row["access_count"]
                })
            
            return results
    
    def get_memory_size_distribution(self) -> Dict[str, int]:
        """Get distribution of memory sizes."""
        with self.store._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT 
                    CASE 
                        WHEN LENGTH(value) < 1000 THEN 'Small (<1KB)'
                        WHEN LENGTH(value) < 10000 THEN 'Medium (1-10KB)'
                        WHEN LENGTH(value) < 100000 THEN 'Large (10-100KB)'
                        ELSE 'Very Large (>100KB)'
                    END as size_category,
                    COUNT(*) as count
                FROM memory_store
                GROUP BY size_category
            """)
            
            return {row["size_category"]: row["count"] for row in cursor.fetchall()}
    
    def get_activity_timeline(self, days: int = 7) -> Dict[str, int]:
        """Get memory activity over time."""
        with self.store._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT 
                    DATE(updated_at) as date,
                    COUNT(*) as updates
                FROM memory_store
                WHERE updated_at >= datetime('now', ? || ' days')
                GROUP BY DATE(updated_at)
                ORDER BY date
            """, (f"-{days}",))
            
            return {row["date"]: row["updates"] for row in cursor.fetchall()}
    
    def get_namespace_statistics(self) -> List[Dict[str, Any]]:
        """Get statistics per namespace."""
        with self.store._get_connection() as conn:
            cursor = conn.cursor()
            
            cursor.execute("""
                SELECT 
                    namespace,
                    COUNT(*) as count,
                    AVG(access_count) as avg_access,
                    MAX(updated_at) as last_update
                FROM memory_store
                GROUP BY namespace
                ORDER BY count DESC
            """)
            
            results = []
            for row in cursor.fetchall():
                results.append({
                    "namespace": row["namespace"],
                    "count": row["count"],
                    "avg_access": round(row["avg_access"] or 0, 2),
                    "last_update": row["last_update"]
                })
            
            return results
    
    def generate_dashboard_report(self) -> Dict[str, Any]:
        """Generate comprehensive dashboard report."""
        return {
            "overview": self.store.get_stats(),
            "top_accessed": self.get_top_accessed_memories(5),
            "size_distribution": self.get_memory_size_distribution(),
            "namespace_stats": self.get_namespace_statistics(),
            "recent_activity": self.get_activity_timeline(7)
        }

# Test analytics
analytics = MemoryAnalytics(pg_store)

print("=" * 70)
print("Lab 4.1: Memory Analytics")
print("=" * 70 + "\n")

# Top accessed
print("📊 Top Accessed Memories:")
top = analytics.get_top_accessed_memories(3)
for i, item in enumerate(top, 1):
    print(f"   {i}. {item['key']} (accessed {item['access_count']} times)")

# Size distribution
print("\n📏 Memory Size Distribution:")
sizes = analytics.get_memory_size_distribution()
for category, count in sizes.items():
    print(f"   {category}: {count} items")

# Namespace stats
print("\n🗂️ Namespace Statistics:")
ns_stats = analytics.get_namespace_statistics()
for stat in ns_stats:
    print(f"   {stat['namespace']}: {stat['count']} items, avg {stat['avg_access']} accesses")

print("\n✅ Memory analytics for monitoring and optimization!")

---
# Part 5: Performance Optimization ⚡

**Production Critical:** Fast memory access at scale

## Lab 5.1: Caching Layer

In [ ]:
from collections import OrderedDict
import time

class LRUCache:
    """Least Recently Used cache implementation."""
    
    def __init__(self, capacity: int = 100):
        self.cache = OrderedDict()
        self.capacity = capacity
        self.hits = 0
        self.misses = 0
    
    def get(self, key: str) -> Optional[Any]:
        """Get item from cache."""
        if key in self.cache:
            self.hits += 1
            # Move to end (most recently used)
            self.cache.move_to_end(key)
            return self.cache[key]
        
        self.misses += 1
        return None
    
    def put(self, key: str, value: Any):
        """Add item to cache."""
        if key in self.cache:
            # Update existing
            self.cache.move_to_end(key)
        else:
            # Add new
            if len(self.cache) >= self.capacity:
                # Remove least recently used
                self.cache.popitem(last=False)
        
        self.cache[key] = value
    
    def invalidate(self, key: str):
        """Remove item from cache."""
        if key in self.cache:
            del self.cache[key]
    
    def clear(self):
        """Clear all cache."""
        self.cache.clear()
        self.hits = 0
        self.misses = 0
    
    def get_stats(self) -> Dict[str, Any]:
        """Get cache statistics."""
        total = self.hits + self.misses
        hit_rate = (self.hits / total * 100) if total > 0 else 0
        
        return {
            "size": len(self.cache),
            "capacity": self.capacity,
            "hits": self.hits,
            "misses": self.misses,
            "hit_rate": f"{hit_rate:.1f}%"
        }

class CachedMemoryStore:
    """Memory store with caching layer."""
    
    def __init__(self, store: PostgreSQLMemoryStore, cache_size: int = 100):
        self.store = store
        self.cache = LRUCache(cache_size)
    
    def _make_cache_key(self, namespace: Tuple[str, ...], key: str) -> str:
        """Create cache key."""
        return f"{':'.join(namespace)}:{key}"
    
    def get(self, namespace: Tuple[str, ...], key: str) -> Optional[Dict[str, Any]]:
        """Get with caching."""
        cache_key = self._make_cache_key(namespace, key)
        
        # Try cache first
        cached = self.cache.get(cache_key)
        if cached is not None:
            return cached
        
        # Cache miss - get from store
        data = self.store.get(namespace, key)
        
        if data:
            # Cache for future
            self.cache.put(cache_key, data)
        
        return data
    
    def put(self, namespace: Tuple[str, ...], key: str, value: Dict[str, Any]):
        """Put with cache invalidation."""
        cache_key = self._make_cache_key(namespace, key)
        
        # Save to store
        self.store.put(namespace, key, value)
        
        # Update cache
        self.cache.put(cache_key, {"value": value})
    
    def delete(self, namespace: Tuple[str, ...], key: str):
        """Delete with cache invalidation."""
        cache_key = self._make_cache_key(namespace, key)
        
        # Delete from store
        self.store.delete(namespace, key)
        
        # Invalidate cache
        self.cache.invalidate(cache_key)
    
    def get_cache_stats(self) -> Dict[str, Any]:
        """Get cache performance stats."""
        return self.cache.get_stats()

# Test cached store
cached_store = CachedMemoryStore(pg_store, cache_size=50)

print("=" * 70)
print("Lab 5.1: Caching Layer")
print("=" * 70 + "\n")

# Save some data
cached_store.put(
    namespace=("test",),
    key="item_1",
    value={"data": "value1"}
)

# First access (cache miss)
start = time.time()
data1 = cached_store.get(("test",), "item_1")
time1 = time.time() - start

# Second access (cache hit)
start = time.time()
data2 = cached_store.get(("test",), "item_1")
time2 = time.time() - start

print(f"First access (cache miss): {time1*1000:.2f}ms")
print(f"Second access (cache hit): {time2*1000:.2f}ms")
print(f"Speedup: {time1/time2:.1f}x faster\n")

# Access multiple times
for i in range(10):
    cached_store.get(("test",), "item_1")

# Check cache stats
stats = cached_store.get_cache_stats()
print("📊 Cache Statistics:")
print(f"   Size: {stats['size']}/{stats['capacity']}")
print(f"   Hits: {stats['hits']}")
print(f"   Misses: {stats['misses']}")
print(f"   Hit Rate: {stats['hit_rate']}")

print("\n✅ Caching significantly improves performance!")

---
# Summary: Production-Ready Memory Systems

## Architecture Patterns

### 1. Storage Backend Selection
```python
# Development
store = InMemoryStore()  # Fast, but not persistent

# Production - Structured Data
store = PostgreSQLMemoryStore()  # ACID, transactions
store = RedisMemoryStore()  # Fast, distributed

# Production - Semantic Search
store = FAISSVectorStore()  # Dense vectors
store = PineconeStore()  # Managed vector DB
```

### 2. Layered Architecture
```
┌─────────────────────────────────┐
│   Application Layer             │
├─────────────────────────────────┤
│   Caching Layer (LRU/Redis)     │
├─────────────────────────────────┤
│   Memory Manager                │
├─────────────────────────────────┤
│   Storage Backend               │
│   (PostgreSQL/MongoDB/Redis)    │
└─────────────────────────────────┘
```

## Best Practices Checklist

### Storage
✅ Use persistent backend in production  
✅ Implement proper indexing  
✅ Add monitoring and alerting  
✅ Regular backups  

### Performance
✅ Add caching layer (Redis/LRU)  
✅ Optimize hot paths  
✅ Batch operations when possible  
✅ Use connection pooling  

### Lifecycle
✅ Automated cleanup jobs  
✅ Archival strategy  
✅ Retention policies  
✅ Health monitoring  

### Multi-Tenancy
✅ Strict namespace isolation  
✅ Per-tenant quotas  
✅ Usage tracking  
✅ Fair resource allocation  

### Security
✅ Access control per tenant  
✅ Encrypt sensitive data  
✅ Audit logging  
✅ Rate limiting  

## Performance Targets

| Operation | Without Cache | With Cache | Target |
|-----------|--------------|------------|--------|
| GET | 5-20ms | <1ms | <2ms |
| PUT | 10-30ms | N/A | <50ms |
| SEARCH | 50-200ms | 5-20ms | <100ms |

## Monitoring Metrics

### Key Metrics to Track:
- **Memory count** (total, per tenant, per namespace)
- **Access patterns** (hot/cold data)
- **Cache hit rate** (target: >80%)
- **Response times** (p50, p95, p99)
- **Storage size** (growth rate)
- **Error rates**

## Deployment Patterns

### Small Scale (<1K users)
```python
# Single PostgreSQL + In-memory cache
store = PostgreSQLMemoryStore()
cached = CachedMemoryStore(store, cache_size=1000)
```

### Medium Scale (1K-100K users)
```python
# PostgreSQL + Redis cache + Vector store
structured = PostgreSQLMemoryStore()
cache = RedisCache()
vectors = FAISSVectorStore()
hybrid = HybridMemorySystem(structured, cache, vectors)
```

### Large Scale (>100K users)
```python
# Sharded PostgreSQL + Redis cluster + Managed vector DB
structured = ShardedPostgreSQLStore(shards=8)
cache = RedisCluster()
vectors = PineconeStore()  # Managed, scalable
hybrid = DistributedMemorySystem(structured, cache, vectors)
```

## Common Pitfalls to Avoid

❌ Using in-memory store in production  
❌ No cleanup/archival strategy  
❌ Missing indexes on frequently queried fields  
❌ No caching layer  
❌ Storing large objects without compression  
❌ No tenant isolation  
❌ Unlimited memory growth  
❌ No monitoring or alerting  

## Next Steps

1. **Implement monitoring dashboard**
2. **Set up automated cleanup jobs**
3. **Load test your memory system**
4. **Implement disaster recovery**
5. **Add comprehensive logging**

---

**Remember:** A well-architected memory system is crucial for production AI agents!